# Logarithmic mean divisia index (LMDI) application on European Union carbon emissions from 2000 to 2017

$$\text { Kaya identity : } C = P \times \frac{G}{P} \times \frac{E}{G} \times \frac{C}{E} = P \times Gp \times EI \times CI $$


$$ \text { P : Population}$$
$$ \text { G : GDP}$$
$$ \text { E : Total energy consumption}$$  
$$ \text { C : } CO_{2} \text {emission due to fossil energy}$$
$$ \text { Gp : GDP per capita}$$
$$ \text { EI : Energy intensity}$$
$$ \text { CI : Carbon intensity}$$

$$ \text{Additive decomposition Change scheme LMDI formulas : }$$

\begin{aligned}
&\Delta C_{t o t}=C^{T}-C^{0}=\Delta C_{\text {P}}+\Delta C_{Gp}+\Delta C_{EI}+\Delta C_{\text {CI}} \\
&\Delta C_{P}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{P^{T}}{P^{0}}\right)\\
&\Delta C_{Gp}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{Gp^{T}}{Gp^{0}}\right)\\
&\Delta C_{EI}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{EI^{T}}{EI^{0}}\right)\\
&\Delta C_{CI}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{CI^{T}}{CI^{0}}\right)\\
\end{aligned}

### Variables

In [1]:
import pandas as pd
data_1 = pd.read_csv("european_dataset_1.csv", index_col='years')
data_2 = pd.read_csv("european_dataset_2.csv", index_col='years')
pop = data_1['pop']
emi = data_1['fossil_emi']
gdp_cap = data_2['gdp_cap']
ei = data_2['ei_total'] # energy intensity of fossil energies (coal, oil, gas)
ci = data_2['ci_total'] # carbon intensity of fossil energies (coal, oil, gas)

var_list = [pop, gdp_cap, ei, ci]

In [15]:
ci

years
1990    2.554797
1991    2.534308
1992    2.514718
1993    2.455896
1994    2.453128
1995    2.414226
1996    2.399772
1997    2.371348
1998    2.350573
1999    2.327864
2000    2.320595
2001    2.305512
2002    2.296141
2003    2.302863
2004    2.281126
2005    2.259205
2006    2.248207
2007    2.250087
2008    2.201740
2009    2.162654
2010    2.141406
2011    2.133732
2012    2.126345
2013    2.093323
2014    2.058384
2015    2.052536
2016    2.030512
2017    2.012290
Name: ci_total, dtype: float64

### Indice 0 and T (T>0) correspond respectively to the base year and the target year. 4 intervals of time are taken into account.
    1 : [2000 - 2004]
    2 : [2005 - 2009]
    3 : [2010 - 2013]
    4 : [2014 - 2017]
    5 : [2000 - 2017]

In [2]:
#intervals
inter_1 = [2000, 2004]
inter_2 = [2005, 2009]
inter_3 = [2010, 2013]
inter_4 = [2014, 2017]
inter_5 = [2000, 2017]

In [3]:
import numpy as np

def from_list_to_pandas(e, p):
    df = pd.DataFrame([e, p], columns = ['pop effect', 'gdp cap effect', 'energy intensity effect',
                                        'carbon intensity effect', 'total effect'])
    df = df.transpose()
    df.columns=['Effect on CO2 emissions [Mtoe]', 'Percentage']
    return df

"""
This function gives the effect of a variable on the variation of the CO2 emissions.
"""
def rate_effect(emi, var):
    return ((emi[1]-emi[0])/(np.log(emi[1])-np.log(emi[0])))*np.log(var[1]/var[0])

"""
Retreive the value of a variable and the carbon emissions for a corresponding interval of time.
"""
def get_target_base_value(interval, emission, variable):
    emi = list(); var = list()

    for ind in interval:
        emi.append(emission[ind])
        var.append(variable[ind])
    return emi, var


"""
This function retreives the effect, on the variation of the CO2 emissions, of each variables on 
these emissions in European Union.
"""
def get_lmdi(var_list, emi, interval):
    list_effect = list()
    for i in range(len(var_list)):
        emi_, var_ = get_target_base_value(interval, emi, var_list[i])
        list_effect.append(rate_effect(emi_, var_))

    list_effect.append(sum(list_effect))
    list_percentage_effect = list()
    for effect in list_effect:
    
        list_percentage_effect.append((effect/np.abs(list_effect[len(list_effect)-1]))*100)
        
    x = np.abs(list_percentage_effect[len(list_effect)-1])
    list_percentage_effect[len(list_effect)-1] = x
    df = from_list_to_pandas(list_effect, list_percentage_effect)
    
    return df


In [4]:
lmdi_1 = get_lmdi(var_list, emi, inter_1)
lmdi_1

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,41.462923,26.750273
gdp cap effect,240.639996,155.251610
energy intensity effect,-61.316241,-39.558865
carbon intensity effect,-65.786678,-42.443018
total effect,155.000000,100.000000


The change in population has caused an **increase of 41.26 MtCO2** compared to the year 2000. While the change in energy intensity resulted in a **decrease of 61.31 MtCO2** compared to the year 2000.

In [5]:
lmdi_2 = get_lmdi(var_list, emi, inter_2)
lmdi_2

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,55.187049,12.894170
gdp cap effect,34.337535,8.022789
energy intensity effect,-357.193370,-83.456395
carbon intensity effect,-160.331214,-37.460564
total effect,-428.000000,100.000000


In [6]:
lmdi_3 = get_lmdi(var_list, emi, inter_3)
lmdi_3

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,13.558499,5.194827
gdp cap effect,41.882584,16.046967
energy intensity effect,-238.527392,-91.389805
carbon intensity effect,-77.913691,-29.851989
total effect,-261.000000,100.000000


In [7]:
lmdi_4 = get_lmdi(var_list, emi, inter_4)
lmdi_4

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,25.451938,82.103025
gdp cap effect,190.689410,615.127130
energy intensity effect,-114.288843,-368.673688
carbon intensity effect,-70.852505,-228.556467
total effect,31.000000,100.000000


In [8]:
lmdi_5 = get_lmdi(var_list, emi, inter_5)
lmdi_5

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,166.291500,27.083306
gdp cap effect,669.442603,109.029740
energy intensity effect,-959.093851,-156.204210
carbon intensity effect,-490.640251,-79.908836
total effect,-614.000000,100.000000


In [9]:
"""
In order to vizualize the effect of the varibles on carbon emissions, the pourcentage obtained with
the function get_lmdi are grouped in function of the different variables. 
"""
def lmdi_variables(lmdi_list, dict_, names, column):
    for name in names:
        temp = list()
        for lmdi in lmdi_list:
            temp.append(lmdi[column][name])
        dict_[name] = temp
    return pd.DataFrame.from_dict(dict_)

names = ['pop effect', 'gdp cap effect', 'energy intensity effect', 'carbon intensity effect', 'total effect']
dict_ = { i : [] for i in names }
lmdi_list = [lmdi_1, lmdi_2, lmdi_3, lmdi_4, lmdi_5]

lmdi_var = lmdi_variables(lmdi_list, dict_, names, 'Percentage')
lmdi_var

,pop effect,gdp cap effect,energy intensity effect,carbon intensity effect,total effect
0,26.750273,155.251610,-39.558865,-42.443018,100.0
1,12.894170,8.022789,-83.456395,-37.460564,100.0
2,5.194827,16.046967,-91.389805,-29.851989,100.0
3,82.103025,615.127130,-368.673688,-228.556467,100.0
4,27.083306,109.029740,-156.204210,-79.908836,100.0


## VISUALIZATION

In [13]:
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)

months = ['2000-2004', '2005-2009', '2010-2013', '2014-2017', '2000-2017']

fig = go.Figure()

fig.add_trace(go.Bar(x=months, y=lmdi_var['pop effect'], name='Population effect',
                     marker_color='indianred'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['gdp cap effect'], name='Gdp per cap effect',
                     marker_color='Yellow'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['energy intensity effect'], name='Energy intensity effect',
                     marker_color='lightsalmon'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['carbon intensity effect'], name='Carbon intensity effect',
                     marker_color='Blue'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['total effect'], name='Total effect',
                     marker_color='Green'))

fig.update_yaxes(title_text="Effect [%]")
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title ="Decomposition of the changes in CO2 emissions in European Union : 2000–2017")
fig.show()